# Tensorflow Example

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import mlflow
from dotenv import load_dotenv
import os
import boto3

In [6]:
# Settting up environment variables

load_dotenv()  

S3_BUCKET = os.getenv("S3_BUCKET")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

In [7]:
boto3.setup_default_session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

NameError: name 'boto3' is not defined

# Prearing the dataset

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
 "mnist",
 split=["train", "test"],
 shuffle_files=True,
 as_supervised=True,
 with_info=True,
)
# Normalizes images: `uint8` -> `float32`
def normalize_img(image, label):
 return tf.cast(image, tf.float32) / 255., label
 
# Train Dataset
ds_train = ds_train.map(
 normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
# Test Dataset
ds_test = ds_test.map(
 normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 4/4 [00:00<00:00, 28.30 url/s]
Extraction completed...: 0 file [00:00, ? file/s]
Dl Completed...: 100%|██████████| 4/4 [00:00<00:00, 23.67 url/s]


Dataset mnist downloaded and prepared to ~\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


# Defining the model

In [3]:
# Define the layers
inputs = tf.keras.Input(shape=(28, 28, 1))
hidden = tf.keras.layers.Flatten()(inputs)
hidden2 = tf.keras.layers.Dense(128, activation="relu")(hidden)
outputs = tf.keras.layers.Dense(10, activation="softmax")(hidden2)
# Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.002)
# Create a Model object
model = tf.keras.Model(inputs, outputs)
# Compile the model
model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [11]:
BATCH_SIZE = 128
EPOCHS = 10
LOGGING_STEPS = 10

EXPERIMENT = "MNIST"
RUN_NAME = "Run_4"

In [16]:
mlflow.tensorflow.autolog(every_n_iter=LOGGING_STEPS)
mlflow.set_experiment(EXPERIMENT)
mlflow.start_run(run_name=RUN_NAME)

<ActiveRun: >

In [17]:
history = model.fit(ds_train, epochs=EPOCHS, validation_data=ds_test,
    batch_size=BATCH_SIZE,
    use_multiprocessing = True)
mlflow.end_run()

Epoch 1/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0030 - accuracy: 0.9991 - val_loss: 0.1587 - val_accuracy: 0.9769
Epoch 2/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0034 - accuracy: 0.9987 - val_loss: 0.1509 - val_accuracy: 0.9772
Epoch 3/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0037 - accuracy: 0.9988 - val_loss: 0.1334 - val_accuracy: 0.9791
Epoch 4/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0055 - accuracy: 0.9983 - val_loss: 0.1597 - val_accuracy: 0.9761
Epoch 5/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0052 - accuracy: 0.9985 - val_loss: 0.1457 - val_accuracy: 0.9795
Epoch 6/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0011 - accuracy: 0.9997 - val_loss: 0.1423 - val_accuracy: 0.9797
Epoch 7/10
469/469 [==============================] - 2s 4ms/step - loss: 4.8705e-04 - accuracy: 0.9999 - val_loss: 0.1379 - val_accuracy: 0.9806
Ep

INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpo3kr5t0f\model\data\model\assets
